# Reto 03-C-Embeddings 

## 1. Descripción General 

En el último desafío (`03-B-Chunking`), trabajamos para entender los límites de tokens con LLM y utilizar la segmentación. Ahora, si hay gigabytes de datos, también tendremos muchos fragmentos por crear. ¿Hay alguna manera de seleccionar los fragmentos de texto más relevantes? La respuesta es sí. Para resolver este problema, podemos observar un proceso llamado Embeddings. Embeddings nos ayuda a crear representaciones numéricas para todos los fragmentos. Luego, podemos encontrar los fragmentos más similares en la lista de embeddings. Una forma popular de encontrar los fragmentos similares es mediante la similitud del coseno.

### **Descripción general de Embeddings**

Un embedding es un formato especial de representación de datos que puede ser fácilmente utilizado por modelos y algoritmos de machine learning. El embedding es una representación densa en información del significado semántico de un fragmento de texto. Cada embedding es un vector de números de punto flotante, de modo que la distancia entre dos embeddings en el espacio vectorial está correlacionada con la similitud semántica entre dos entradas en el formato original. Por ejemplo, si dos textos son similares, entonces sus representaciones vectoriales también deberían ser similares.

Diferentes modelos de embedding de Azure OpenAI están específicamente creados para ser buenos en tareas particulares:
- Los embeddings de similitud son buenos para capturar la similitud semántica entre dos o más fragmentos de texto.
- Los embeddings de búsqueda de texto ayudan a encontrar qué documento largo es relevante para una consulta corta.
- Los embeddings de búsqueda de código son útiles para incrustar fragmentos de código y consultas de búsqueda en lenguaje natural.

Los embeddings facilitan hacer machine learning en grandes entradas que representan palabras al capturar las similitudes semánticas en un espacio vectorial. Por lo tanto, podemos usar embeddings para determinar si dos fragmentos de texto están semánticamente relacionados o son similares, y de manera inherente proporcionar una puntuación para evaluar la similitud.

### **Similitud del Coseno**
Un enfoque utilizado anteriormente para emparejar documentos similares se basaba en contar el número máximo de palabras comunes entre documentos. Esto es defectuoso ya que, a medida que aumenta el tamaño del documento, aumenta la superposición de palabras comunes incluso si los temas difieren. Por lo tanto, la similitud del coseno es un mejor enfoque.

Matemáticamente, la similitud del coseno mide el coseno del ángulo entre dos vectores proyectados en un espacio multidimensional. Esto es beneficioso porque si dos documentos están muy separados por la distancia euclidiana debido al tamaño, aún podrían tener un ángulo más pequeño entre ellos y, por lo tanto, una mayor similitud del coseno.

Los embeddings de Azure OpenAI se basan en la similitud del coseno para calcular la similitud entre documentos y una consulta.

### **Aplicaciones**

Se pueden crear embeddings para todos los diferentes tipos de datos, incluyendo imágenes, audio, video y texto. En este notebook, veremos la generación de embeddings para archivos de texto y CSV.

Hay muchas aplicaciones en las que los embeddings pueden ser útiles. Por ejemplo, digamos que quieres clasificar un fragmento de texto. Una vez que se generan los embeddings, se pueden insertar en un modelo de aprendizaje automático para predecir la etiqueta correcta. Además, puedes utilizar embeddings para la similitud en datos de series temporales, datos de gráficos o para perfiles de usuarios o productos. Un caso de uso muy popular es el que implica la búsqueda semántica. Si deseas recuperar documentos que sean muy relevantes para tu consulta, se pueden generar embeddings tanto para la consulta como para los documentos con el fin de obtener una respuesta precisa. Veremos un ejemplo de esto en el Desafío 4.

## 2. Comencemos con la Implementación

Necesitarás importar los módulos necesarios. Las siguientes celdas son pasos de configuración de claves que ya completaste en los desafíos anteriores.

In [ ]:
! pip install num2words
! pip install plotly
! pip install "openai==0.28.1" 

In [ ]:
import openai
import os
import re 
import requests
import sys
from num2words import num2words 
import pandas as pd 
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity 
import tiktoken
from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt
load_dotenv() 

Configura tu entorno para acceder a tus claves de Azure OpenAI. Consulta tu recurso de Azure OpenAI en el Portal de Azure para obtener información sobre tu punto de conexión y claves de Azure OpenAI.

Por razones de seguridad, almacena tu información sensible en un archivo .env.

In [ ]:
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
embedding_model=os.getenv("EMBEDDING_MODEL_NAME")

## 3. Generar Embeddings en texto

#### Tarea #1 del Estudiante:
Utiliza la clase Azure OpenAI Embeddings para crear un embedding para el texto de entrada mostrado a continuación.

In [ ]:

input="I would like to order a pizza"

# Agrega código aquí

El método `openai.Embedding.create()` tomará una lista de texto - aquí tenemos una sola oración - y luego devolverá una lista que contiene un solo embedding. Puedes utilizar estos embeddings al buscar, proporcionar recomendaciones, clasificación y más.

### 3.1 Generar Embeddings para un archivo CSV

#### Tarea #2 del Estudiante:
Ingresa la ruta del archivo `Automobile.csv` que puedes encontrar en la carpeta `/data`. Ejecuta las celdas a continuación.

In [ ]:
df=pd.read_csv(os.path.join(os.getcwd(),r'Enter path here'))
df

In [ ]:
shortened_df = df[['name', 'mpg', 'origin']]
shortened_df

In [ ]:
tokenizer = tiktoken.get_encoding("cl100k_base")
shortened_df['n_tokens'] = shortened_df["name"].apply(lambda x: len(tokenizer.encode(x)))
shortened_df = shortened_df[shortened_df.n_tokens<8192]
len(shortened_df)

In [ ]:
shortened_df

In [ ]:
sample_encode = tokenizer.encode(shortened_df.name[0]) 
decode = tokenizer.decode_tokens_bytes(sample_encode)
decode

In [ ]:
len(decode)
shortened_df['ada-v2'] = shortened_df['name'].apply(lambda x : get_embedding(x, engine = embedding_model)) 

In [ ]:
shortened_df

Los embeddings generados a partir del archivo CSV pueden usarse para realizar búsquedas. Puedes calcular la similitud del coseno entre un embedding de consulta y los embeddings del archivo CSV. Luego, puedes clasificar los resultados de la búsqueda según lo más relevante para la consulta. Veremos una aplicación de embeddings en el Desafío 4.

## Criterios de Éxito

Para completar este desafío con éxito:

* Demuestra comprensión de embeddings trabajando con diferentes entradas.